In [1]:
import random
import os
import numpy as np
from tqdm import trange
from pyproj import CRS
from pyproj import Transformer
from tms2geotiff.tms2geotiff import draw_tile

from shapely.geometry import Point
import geopandas as gpd

# Load Geofabrik OSM Landuse

In [2]:
tms_crs = 'EPSG:3857'

In [3]:
landused_geofabrik_shp_path = 'gis_osm_landuse_a_free_1.shp'
landuse_gdf = gpd.read_file(landused_geofabrik_shp_path)
landuse_gdf = landuse_gdf.to_crs(tms_crs)
landuse_gdf.head()

,osm_id,code,fclass,name,geometry
0,3870564,7218,grass,NaN,"POLYGON ((1847748.602 6257290.611, 1847753.667..."
1,3870580,7201,forest,Tajvan,"POLYGON ((1565314.545 6231141.063, 1565366.598..."
2,3870632,7201,forest,NaN,"POLYGON ((1851334.247 6257024.145, 1851340.548..."
3,4072815,7202,park,NaN,"POLYGON ((1839947.365 6304903.295, 1839952.163..."
4,4072816,7202,park,NaN,"POLYGON ((1839979.915 6304858.187, 1839980.661..."


In [4]:
landuse_class_name = "farmland"
class_gdf = landuse_gdf[landuse_gdf.fclass == landuse_class_name]
class_gdf.head()

,osm_id,code,fclass,name,geometry
15,4549853,7229,farmland,NaN,"POLYGON ((1846297.185 6250296.116, 1846323.935..."
106,9370259,7229,farmland,NaN,"POLYGON ((1449272.759 6305686.502, 1449291.160..."
107,9370286,7229,farmland,NaN,"POLYGON ((1452859.016 6303583.134, 1452978.740..."
168,16944594,7229,farmland,NaN,"POLYGON ((1613877.684 6469433.948, 1613883.795..."
169,16944597,7229,farmland,NaN,"POLYGON ((1613224.650 6469451.662, 1613231.663..."


# Make a list with geometries and there areas

In [5]:
total_area = 0
geometry_list = []
area_list = []
p = None
for geometry in class_gdf.geometry:
    geometry_list.append(geometry)
    
    area = geometry.area
    area_list.append(area)
    total_area += area
    
relative_area_list = [i / total_area for i in area_list]
print('Total area', int(total_area / (1000 * 1000)), 'sq.km.')

Total area 61484 sq.km.


# Make functions to download images and get random points

In [6]:
def download_tms_as_tiff(source, pt1, pt2, zoom, dist='t.tif'):
    image = draw_tile(source, pt1[0], pt1[1], pt2[0], pt2[1],
                      zoom, dist)
    return image

In [7]:
def random_point_inside_polygon(polygon, max_iterations=1024):
    # https://gis.stackexchange.com/questions/207731/generating-random-coordinates-in-multipolygon-in-python
    minx, miny, maxx, maxy = polygon.bounds
    for i in range(max_iterations):
        x, y = random.uniform(minx, maxx), random.uniform(miny, maxy)
        point = Point(x, y)
        if polygon.contains(point):
            return x, y
    return None

# Download imagery

In [10]:
dx, dy = 917 *2, 917 *2 # -> 768x768 output
zoom_level = 16

xys_source_list = [
    'put your XYZ Links',
    'put your XYZ Links',
]

geometry_crs_epsg = 3857
download_function_crs_epsg = 4326

download_path = 'myclass_czech_5120'

total_count = 5120

In [12]:
count = 1
while count +1 < total_count:
    
    # Get random xyz source
    source = random.choice(xys_source_list)
    
    # Choose random polygon weighted by area
    weighted_random_n = np.random.choice([i for i in range(len(relative_area_list))],
                                            size = 1,
                                            p=relative_area_list)[0]
    polygon = geometry_list[weighted_random_n]
    
    # Choose random point inside a polygon
    xy = random_point_inside_polygon(polygon, max_iterations=1024)
    if xy is None:
        print('Could not find point inside a polygon. Max iteration count reached.')
        continue
    x1, y1 = random_point_inside_polygon(polygon, max_iterations=1024)
    x1, y1 = x1 - (0.5 * dx), y1 - (0.5 * dy)
    
    # Make a rectangle to download
    x2, y2 = x1 + dx, y1 + dy
    transformer = Transformer.from_crs(geometry_crs_epsg, download_function_crs_epsg)
    pt1 = transformer.transform(x1, y1)
    pt2 = transformer.transform(x2, y2)
    
    fn = f'{count}.tif'
    fp = os.path.join(download_path, fn)
    _ = download_tms_as_tiff(source, pt1, pt2, zoom_level, dist=fp)

    print('Done: ', count)
    count += 1

print('All done!')

16it [00:00, 31.18it/s]


Done:  1


16it [00:01, 13.69it/s]


Done:  2


16it [00:00, 31.29it/s]


...

Done:  5


16it [00:00, 30.95it/s]


Done:  6


10it [00:01,  9.43it/s]


KeyboardInterrupt: 